### 🧠 What is Query Decomposition?
Query decomposition is the process of taking a complex, multi-part question and breaking it into simpler, atomic sub-questions that can each be retrieved and answered individually.

#### ✅ Why Use Query Decomposition?

- Complex queries often involve multiple concepts

- LLMs or retrievers may miss parts of the original question

- It enables multi-hop reasoning (answering in steps)

- Allows parallelism (especially in multi-agent frameworks)

In [2]:
from langchain.chat_models import init_chat_model
from langchain.prompts import PromptTemplate
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain.chains.combine_documents import create_stuff_documents_chain 

In [3]:
# Step 1: Load and embed the document
loader = TextLoader("langchain_crewai_dataset.txt")
docs = loader.load()

splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=50)
chunks = splitter.split_documents(docs)

embedding = HuggingFaceEmbeddings(model_name="D:\\model\\BAAI-bge-m3")
vectorstore = FAISS.from_documents(chunks, embedding)
retriever = vectorstore.as_retriever(search_type="mmr", search_kwargs={"k": 4, "lambda_mult": 0.7})

In [4]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY")

llm=init_chat_model(model="groq:meta-llama/llama-4-maverick-17b-128e-instruct")
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x00000218935B06E0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x00000218935B12B0>, model_name='meta-llama/llama-4-maverick-17b-128e-instruct', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [18]:
# Step 3: Query decomposition
decomposition_prompt = PromptTemplate.from_template("""
You are an AI assistant. Decompose the following complex question into 2 to 4 smaller sub-questions for better document retrieval.

Question: "{question}"

Sub-questions:
""")
decomposition_chain = decomposition_prompt | llm | StrOutputParser()

In [19]:
query = "How does LangChain use memory and agents compared to CrewAI?"
decomposition_question=decomposition_chain.invoke({"question": query})


In [20]:
print(decomposition_question)

To tackle the complex question, "How does LangChain use memory and agents compared to CrewAI?" we can break it down into smaller, more manageable sub-questions. Here's a decomposition into 3 sub-questions that can facilitate better document retrieval:

1. **What is LangChain and how does it utilize memory and agents?**
   - This sub-question aims to understand the core functionalities of LangChain, particularly focusing on its use of memory and agents. It will help in gathering information on how LangChain operates.

2. **What is CrewAI and how does it utilize memory and agents?**
   - Similarly, this sub-question seeks to understand CrewAI's core functionalities, especially its use of memory and agents. It will provide insights into how CrewAI operates.

3. **What are the differences in the utilization of memory and agents between LangChain and CrewAI?**
   - This sub-question directly compares the two, focusing on the differences in their approaches to using memory and agents. It wil

In [21]:
# Step 4: QA chain per sub-question
qa_prompt = PromptTemplate.from_template("""
Use the context below to answer the question.

Context:
{context}

Question: {input}
""")
qa_chain = create_stuff_documents_chain(llm=llm, prompt=qa_prompt)

In [22]:
# Step 5: Full RAG pipeline logic
def full_query_decomposition_rag_pipeline(user_query):
    # Decompose the query
    sub_qs_text = decomposition_chain.invoke({"question": user_query})
    sub_questions = [q.strip("-•1234567890. ").strip() for q in sub_qs_text.split("\n") if q.strip()] 
    results = []
    for subq in sub_questions:
        docs = retriever.invoke(subq) 
        result = qa_chain.invoke({"input": subq, "context": docs}) 
        results.append(f"Q: {subq}\nA: {result}")
    
    return "\n\n".join(results)

In [23]:
# Step 6: Run
query = "How does LangChain use memory and agents compared to CrewAI?"
final_answer = full_query_decomposition_rag_pipeline(query)
print("✅ Final Answer:\n")
print(final_answer)

✅ Final Answer:

Q: To tackle the complex question, "How does LangChain use memory and agents compared to CrewAI?", we can break it down into smaller, more manageable sub-questions. Here are 2 to 4 sub-questions that can facilitate better document retrieval:
A: To tackle the complex question, "How does LangChain use memory and agents compared to CrewAI?", we can break it down into the following sub-questions:

1. **How does LangChain utilize memory in its workflows?** - This sub-question focuses on understanding the role of memory within LangChain, particularly how it is used across different components like agents and chains.

2. **What is the role of agents in LangChain, and how do they operate?** - This sub-question aims to delve into the specifics of LangChain agents, including their planner-executor model, dynamic decision-making, and how they use tools and memories.

3. **How does CrewAI manage role-based collaboration, and how does it integrate with LangChain agents and tools?**